# Aggregating DataFrames in PySpark HW

First let's start up our PySpark instance

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Read in the dataFrame for this Notebook

In [2]:
airbnb = spark.read.csv('nyc_air_bnb.csv',inferSchema=True,header=True)

## About this dataset

This dataset describes the listing activity and metrics for Air BNB bookers in NYC, NY for 2019. Each line in the dataset is a booking. 

**Source:** https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/data

Let's go ahead and view the first few records of the dataset so we know what we are working with.

In [3]:
airbnb.toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355.0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365.0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194.0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49074,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,None,None,2,9.0
49075,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,None,None,2,36.0
49076,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,None,None,1,27.0
49077,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,None,None,6,2.0


Now print the schema so we can make sure all the variables have the correct types

In [4]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



Notice here that some of the columns that are obviously numeric have been incorrectly identified as "strings". Let's edit that. Otherwise we cannot aggregate any of the numeric columns.

In [42]:
airbnb = airbnb.withColumn("id", airbnb["id"].cast("int")).withColumn("host_id", airbnb["host_id"].cast("int")) \
.withColumn("price", airbnb["price"].cast("int")).withColumn("minimum_nights", airbnb["minimum_nights"].cast("int")) \
.withColumn("number_of_reviews", airbnb["number_of_reviews"].cast("int")).withColumn("calculated_host_listings_count", airbnb["calculated_host_listings_count"].cast("int")) \
.withColumn("availability_365", airbnb["availability_365"].cast("double")).withColumn("latitude", airbnb["latitude"].cast("double")) \
.withColumn("longitude", airbnb["longitude"].cast("double")).withColumn("reviews_per_month", airbnb["reviews_per_month"].cast("double")) \
.withColumn("last_review", airbnb["last_review"].cast("date"))

In [43]:
airbnb.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: date (nullable = true)
 |-- reviews_per_month: double (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: double (nullable = true)



### Alright now we are ready to dig in!


### 1. How many rows are in this dataset?

In [44]:
from pyspark.sql.functions import * 

In [45]:
airbnb.count()

49079

### 2. How many total reviews does each host have?

In [46]:
airbnb.dtypes

[('id', 'int'),
 ('name', 'string'),
 ('host_id', 'int'),
 ('host_name', 'string'),
 ('neighbourhood_group', 'string'),
 ('neighbourhood', 'string'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('room_type', 'string'),
 ('price', 'int'),
 ('minimum_nights', 'int'),
 ('number_of_reviews', 'int'),
 ('last_review', 'date'),
 ('reviews_per_month', 'double'),
 ('calculated_host_listings_count', 'int'),
 ('availability_365', 'double')]

In [88]:
# cau2 = airbnb.groupBy('host_id').sum('number_of_reviews')
cau2a = airbnb.select('host_id', 'host_name', 'number_of_reviews') 
# cau2a.show()
cau2b = cau2a.groupBy('host_id').agg(sum(cau2a.number_of_reviews).alias('Total number of reviews'))

In [89]:
cau2b.show()

+--------+-----------------------+
| host_id|Total number of reviews|
+--------+-----------------------+
|  291112|                     35|
| 1384111|                    103|
| 1597481|                     13|
| 2108853|                     18|
| 2429432|                     27|
| 2530670|                    134|
| 3432742|                      2|
| 1360296|                     13|
| 2124690|                      1|
| 6414252|                      1|
| 9637768|                     47|
| 9947836|                     87|
| 9430366|                    159|
| 7974574|                      5|
| 5907325|                     32|
|13749425|                    198|
| 5771331|                     20|
| 9784206|                    204|
| 4702135|                      0|
|19239110|                     25|
+--------+-----------------------+
only showing top 20 rows



### 3. Show the min and max of all the numeric variables in the dataset

In [75]:
airbnb.summary("min", "max").toPandas()

,summary,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,min,2539,1 Bed Apt in Utopic Williamsburg,2438,"very clean studio app""",194716858,2,-74.16254,-74.24442,-73.90783,-74,0,0,0.0,0,0.0
1,max,36487245,"ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡",274321313,현선,Woodside,Woodside,40.91306,2.4906403E7,Shared room,10000,1250,629,58.5,365,365.0


### 4. Which host had the highest number of reviews?

Only display the top result.

Bonus: format the column names

In [90]:
temp = cau2b.select('Host_id').orderBy(col('Total number of reviews').desc()).limit(1)

In [91]:
temp.show()

+--------+
| Host_id|
+--------+
|37312959|
+--------+



### 5. On average, how many nights did most hosts specify for a minimum?

In [98]:
cau5a = airbnb.summary("mean")

In [99]:
cau5b = cau5a.select('minimum_nights')

In [100]:
cau5b.toPandas()

,minimum_nights
0,7.1286126280910596


### 6. What is the most expensive neighborhood to stay in on average?

Note: only show the one result

In [103]:
airbnb.summary("max").select('neighbourhood').show()

+-------------+
|neighbourhood|
+-------------+
|     Woodside|
+-------------+



### 7. Display a two by two table that shows the average prices by room type (private and shared only) and neighborhood group (Manhattan and Brooklyn only)

In [111]:
cau7a = airbnb.groupBy('room_type').agg(mean(airbnb['price']))

In [118]:
cau7a.filter((col('room_type') == 'Private room') | (col('room_type') == 'Shared room')).toPandas()

,room_type,avg(price)
0,Shared room,70.132988
1,Private room,89.516917


### Alright that's all folks!

### Great job!